In [1]:
# -*- coding: utf-8 -*-

import hashlib
import json
import os
import sqlite3

import requests

bbox_list = []

# get attribute names and descriptions from HERE Platform Data:
# attributes = json.loads(requests.get('https://s.fleet.ls.hereapi.com/1//doc/attributes.json?region=TWN&release=LATEST&apikey={}'.format(apikey)).text)

traffic_sign_category_dict = {
    1: 'REGULATORY TRAFFIC SIGN CATEGORY',
    2: 'SAFETY INFORMATION TRAFFIC SIGN CATEGORY',
    3: 'WARNING TRAFFIC SIGN CATEGORY',
}

traffic_sign_type_dict = {
    1: 'START OF NO OVERTAKING',
    10: 'RAILWAY CROSSING UNPROTECTED',
    11: 'ROAD NARROWS',
    12: 'SHARP CURVE LEFT',
    13: 'SHARP CURVE RIGHT',
    14: 'WINDING ROAD STARTING LEFT',
    15: 'WINDING ROAD STARTING RIGHT',
    16: 'START OF NO OVERTAKING TRUCKS',
    17: 'END OF NO OVERTAKING TRUCKS',
    18: 'STEEP HILL UPWARDS',
    19: 'STEEP HILL DOWNWARDS',
    2: 'END OF NO OVERTAKING',
    20: 'STOP SIGN',
    21: 'LATERAL WIND',
    22: 'GENERAL WARNING',
    23: 'RISK OF GROUNDING',
    24: 'GENERAL CURVE',
    25: 'END OF ALL RESTRICTIONS',
    26: 'GENERAL HILL',
    27: 'ANIMAL CROSSING',
    28: 'ICY CONDITIONS',
    29: 'SLIPPERY ROAD',
    3: 'PROTECTED OVERTAKING - EXTRA LANE',
    30: 'FALLING ROCKS',
    31: 'SCHOOL ZONE',
    32: 'TRAMWAY CROSSING',
    33: 'CONGESTION HAZARD',
    34: 'ACCIDENT HAZARD',
    35: 'PRIORITY OVER ONCOMING TRAFFIC',
    36: 'YIELD TO ONCOMING TRAFFIC',
    37: 'CROSSING WITH PRIORITY FROM THE RIGHT',
    4: 'PROTECTED OVERTAKING - EXTRA LANE RIGHT',
    41: 'PEDESTRIAN CROSSING',
    42: 'YIELD',
    43: 'DOUBLE HAIRPIN',
    44: 'TRIPLE HAIRPIN',
    45: 'EMBANKMENT',
    46: 'TWO-WAY TRAFFIC',
    47: 'URBAN AREA',
    48: 'HUMP BRIDGE',
    49: 'UNEVEN ROAD',
    5: 'PROTECTED OVERTAKING - EXTRA LANE LEFT',
    50: 'FLOOD AREA',
    51: 'OBSTACLE',
    52: 'HORN SIGN',
    53: 'NO ENGINE BRAKE',
    54: 'END OF NO ENGINE BRAKE',
    55: 'NO IDLING',
    56: 'TRUCK ROLLOVER',
    57: 'LOW GEAR',
    58: 'END OF LOW GEAR',
    59: 'BICYCLE CROSSING',
    6: 'LANE MERGING FROM THE RIGHT',
    60: 'YIELD TO BICYCLES',
    61: 'NO TOWED CARAVAN ALLOWED',
    62: 'NO TOWED TRAILER ALLOWED',
    63: 'NO CAMPER OR MOTORHOME ALLOWED',
    64: 'NO TURN ON RED',
    65: 'TURN PERMITTED ON RED',
    7: 'LANE MERGING FROM THE LEFT',
    8: 'LANE MERGE CENTRE',
    9: 'RAILWAY CROSSING PROTECTED',
}

traffic_sign_subcategory = {
    1: 'PRIORITY SIGN'
}


# def pde_tile_index(link_id_list):
#     length_limit = 160
#     counter = len(link_id_list) / length_limit
#     i = 0
#     tile_set = set()
#     while i < counter:
#         link_id_list_split = link_id_list[length_limit * i:length_limit * (i + 1)]
#         pde_index_url = 'https://s.fleet.ls.hereapi.com/1/index.json?layer=ROAD_GEOM_FCn&attributes=LINK_ID&values={}&apikey={}'.format(
#             ','.join(link_id_list_split), apikey)
#         i += 1
#         pde_index_results = json.loads(requests.get(pde_index_url).text)
#         layers = pde_index_results.get('Layers')
#         for layer in layers:
#             layer_name = layer.get('layer')
#             layer_level = layer.get('level')
#             tile_xys = layer.get('tileXYs')
#             for tile_xy in tile_xys:
#                 tile_x = tile_xy.get('x')
#                 tile_y = tile_xy.get('y')
#                 tile_set.add('{},{},{},{}'.format(layer_name, layer_level, tile_x, tile_y))
#     return (tile_set)


def speed_limit_checker(user_id, input_file):
    cursor = conn.cursor()
    trace_counts_checker_sql = "SELECT count(*) FROM trace_points WHERE trace_points.user_id='{}' AND trace_points.file_name='{}'".format(
        user_id, input_file)
    speed_limit_checker_sql = "SELECT count(*) FROM trace_points INNER JOIN route_links ON trace_points.link_id_matched = route_links.link_id WHERE (((route_direction = 'f' AND speed_kph > from_ref_speed_limit + 10 AND from_ref_speed_limit>0)) OR ((route_direction = 't' AND speed_kph > to_ref_speed_limit + 10 AND to_ref_speed_limit>0))) AND trace_points.user_id='{}' AND trace_points.file_name='{}'".format(
        user_id, input_file)
    trace_counts_result = cursor.execute(trace_counts_checker_sql).fetchall().pop()[0]
    overspeed_checker_result = cursor.execute(speed_limit_checker_sql).fetchall().pop()[0]
    print('matched trace_length: {} seconds\toverspeed_time: {} seconds ({}%)'.format(trace_counts_result,
                                                                                      overspeed_checker_result, round(
            (int(overspeed_checker_result) / int(trace_counts_result)) * 100, 2)))
    total_over_speed_time_list.append(overspeed_checker_result)
    total_matched_trace_length_list.append(trace_counts_result)


def warnings_checker(user_id, input_file):
    cursor = conn.cursor()
    warnings_checker_sql = "SELECT * FROM warnings WHERE user_id='{}' AND file_name='{}'".format(user_id, input_file)
    warnings_result = cursor.execute(warnings_checker_sql).fetchall()
    print('{} warnings: {}'.format(len(warnings_result), warnings_result))
    total_warnings_count_list.append(len(warnings_result))
    total_warnings_text_list.append(warnings_result)


def warning_sign_checker(user_id, input_file):
    cursor = conn.cursor()
    warning_sign_checker_sql = "SELECT user_id, file_name, traffic_sign_desc, count(traffic_sign_type) FROM warning_traffic_signs WHERE user_id='{}' AND file_name='{}' group by user_id, file_name, traffic_sign_desc".format(
        user_id, input_file)
    warning_sign_result = cursor.execute(warning_sign_checker_sql).fetchall()
    total_warning_sign_text_list.append(warning_sign_result)


def bbox_generator(user_id, file_name):
    cursor = conn.cursor()
    max_lat_lon_sql = "SELECT MAX(lat_matched), MIN(lat_matched), MAX(lon_matched), MIN(lon_matched) FROM trace_points WHERE trace_points.user_id='{}' AND trace_points.file_name='{}'".format(
        user_id, file_name)
    max_lat, min_lat, max_lon, min_lon = cursor.execute(max_lat_lon_sql).fetchall()[0]
    avg_lat = (max_lat + min_lat) / 2
    avg_lon = (max_lon + min_lon) / 2
    bbox_list.append((max_lat, min_lat, max_lon, min_lon, avg_lat, avg_lon))


def rme_checkers(user_id, input_file):
    rme_url = 'https://m.fleet.ls.hereapi.com/2/matchroute.json?routemode=car&filetype={}&apikey={}&attributes=SPEED_LIMITS_FCn(*),TRAFFIC_SIGN_FCn(*)'.format(file_type, apikey)
    cursor = conn.cursor()
    f = open(input_file, mode='r', encoding='utf-8')
    md5 = hashlib.md5(f.read().encode(encoding='utf-8')).hexdigest()
    print('filename: {}\t|\tchecksum: {}'.format(input_file, md5))
    check_md5 = "select * from file_checksum where md5='{}'".format(md5)
    cursor.execute(check_md5)
    check_md5_result = cursor.fetchall()
    if len(check_md5_result) == 0:
        print('GPS trace is new, uploading RME...')
        insert_md5 = "insert into file_checksum values ('{}','{}','{}','{}')".format(user_id, input_file, file_type, md5)
        cursor.execute(insert_md5)
        conn.commit()
        payload = open(input_file, mode='r', encoding='utf-8').read().encode('utf-8')
        r = requests.post(rme_url, data=payload)
        print('Write result into cache database.')
        r.encoding = 'utf-8'
        print(r.text)
        rme_result = json.loads(r.text)
        rme_result = rme_result_parsing(input_file, file_type, rme_result)
        if str(rme_result) != "(None, None, None)":
            speed_limit_checker(user_id, input_file)
            warnings_checker(user_id, input_file)
            warning_sign_checker(user_id, input_file)
            bbox_generator(user_id, input_file)
        else:
            print("No route matched.")
    else:
        print('GPS trace existed already, querying from cache database.')

        check_cache_exists = "select * from route_links where file_name in (select file_name from file_checksum where md5 = '{}')".format(
            md5)
        query_cache = "select distinct user_id, file_name from file_checksum where md5='{}'".format(md5)
        if len(cursor.execute(check_cache_exists).fetchall()) > 0:
            user_id = cursor.execute(query_cache).fetchall().pop()[0]
            file_name = cursor.execute(query_cache).fetchall().pop()[1]
            speed_limit_checker(user_id, file_name)
            warnings_checker(user_id, file_name)
            warning_sign_checker(user_id, file_name)
            bbox_generator(user_id, input_file)
        else:
            print("No route matched.")
    print('-------')


def rme_result_parsing(file_name, file_type, rme_result):
    file_name = file_name
    file_type = file_type
    route_links = rme_result.get('RouteLinks')
    trace_points = rme_result.get('TracePoints')
    warnings = rme_result.get('Warnings')
    cursor = conn.cursor()
    rme_link_id_results = []
    if rme_result.get('RouteLinks'):
        if len(rme_result.get('RouteLinks')) > 0:
            for route_link in route_links:
                shape = route_link['shape']
                link_id = route_link['linkId']
                rme_link_id_results.append(str(link_id))
                if int(link_id) > 0:
                    route_direction = 'f'
                else:
                    route_direction = 't'
                functional_class = route_link['functionalClass']
                confidence = route_link['confidence']
                link_length = route_link['linkLength']
                m_sec_to_reach_link_from_start = route_link['mSecToReachLinkFromStart']
                if route_link.get('attributes'):
                    for table_name, values in route_link.get('attributes').items():
                        if table_name == 'SPEED_LIMITS_FCN':
                            for value in values:
                                from_ref_speed_limit = value.get('FROM_REF_SPEED_LIMIT')
                                to_ref_speed_limit = value.get('TO_REF_SPEED_LIMIT')
                                if not from_ref_speed_limit:
                                    from_ref_speed_limit = 0
                                if not to_ref_speed_limit:
                                    to_ref_speed_limit = 0
                                route_links_insert_data = "insert into route_links values ('{}',{},'{}','{}','{}',{},{},{},{},{})".format(
                                    file_name, link_id, route_direction, shape, functional_class, confidence, link_length,
                                    m_sec_to_reach_link_from_start, from_ref_speed_limit, to_ref_speed_limit)
                                cursor.execute(route_links_insert_data)
                                conn.commit()
                        elif table_name == 'TRAFFIC_SIGN_FCN':
                            for value in values:
                                traffic_sign_category = value.get('TRAFFIC_SIGN_CATEGORY')
                                if traffic_sign_category == '3':  # WARNING TRAFFIC SIGN CATEGORY
                                    link_ids = value.get('LINK_IDS')
                                    condition_id = value.get('CONDITION_ID')
                                    condition_type = value.get('CONDITION_TYPE')
                                    traffic_sign_type = int(value.get('TRAFFIC_SIGN_TYPE'))
                                    traffic_sign_desc = traffic_sign_type_dict[traffic_sign_type]
                                    insert_traffic_sign = "insert into warning_traffic_signs values ('{}','{}','{}','{}','{}','{}','{}')".format(
                                        user_id, file_name, condition_id, condition_type, link_ids, traffic_sign_type,
                                        traffic_sign_desc)
                                    print(insert_traffic_sign)
                                    cursor.execute(insert_traffic_sign)
                                    conn.commit()

    if rme_result.get('TracePoints'):
        if len(rme_result.get('TracePoints')) > 0:
            i = 0
            while i < len(rme_result.get('TracePoints')):
                trace_point = rme_result.get('TracePoints')[i]
                timestamp = trace_point['timestamp']
                lat = trace_point['lat']
                lon = trace_point['lon']
                lat_matched = trace_point['latMatched']
                lon_matched = trace_point['lonMatched']
                link_id_matched = trace_point['linkIdMatched']
                heading_matched = trace_point['headingMatched']
                speed_mps = trace_point['speedMps']
                speed_kph = float(speed_mps) * 3.6
                trace_points_insert_data = "insert into trace_points values ('{}','{}','{}','{}',{},{},{},{},{},{},{},{})".format(
                    user_id, file_name, file_type, timestamp, link_id_matched, lat, lon, lat_matched, lon_matched,
                    heading_matched, speed_mps, speed_kph)
                cursor.execute(trace_points_insert_data)
                conn.commit()
                i += 1
    if rme_result.get('Warnings'):
        if len(rme_result.get('Warnings')) > 0:
            for warning in warnings:
                route_link_seq_num = warning.get('routeLinkSeqNum')
                trace_point_seq_num = warning.get('tracePointSeqNum')
                category = warning.get('category')
                text = warning.get('text')
                warning_trace_points.append({'tracepoint': rme_result.get('TracePoints')[trace_point_seq_num - 1], 'text': text})
                if int(route_link_seq_num) > 0 and int(trace_point_seq_num) > 0:
                    warnings_insert_data_sql = "insert into warnings values ('{}','{}','{}','{}','{}','{}')".format(
                        user_id, file_name, route_link_seq_num, trace_point_seq_num, category, text)
                    cursor.execute(warnings_insert_data_sql)
                    conn.commit()
    return route_links, trace_points, warnings


if __name__ == '__main__':

    apikey = 'vm_XH415QtO7f6zEHPaVv-jIowwRnBRSHW1bXzBk3SA'  # YOUR APP ID
    conn = sqlite3.connect('rme_safety_checker.sqlite')
    cursor = conn.cursor()
    create_file_checksum = "CREATE TABLE IF NOT EXISTS file_checksum (user_id varchar(255),file_name varchar(255),file_type varchar(255),md5 varchar(255))"
    create_route_links = "CREATE TABLE IF NOT EXISTS route_links (file_name varchar(255),link_id int,route_direction varchar(1),shape varchar(65535),functional_class int, confidence float, link_length float, m_sec_to_reach_link_from_start float, from_ref_speed_limit int, to_ref_speed_limit int)"
    create_trace_point = "CREATE TABLE IF NOT EXISTS trace_points (user_id varchar(255),file_name varchar(255),file_type varchar(255),timestamp int, link_id_matched int,lat float,lon float,lat_matched float,lon_matched float,heading_matched float,speed_mps float,speed_kph float)"
    create_warnings = "CREATE TABLE IF NOT EXISTS warnings (user_id varchar(255), file_name varchar(255), route_link_seq_num int, trace_point_seq_num int, category int, text varchar(255))"
    create_warning_traffic_signs = "CREATE TABLE IF NOT EXISTS warning_traffic_signs (user_id varchar(255), file_name varchar(255), condition_id int, condition_type int, link_ids varchar(255), traffic_sign_type int, traffic_sign_desc varchar(255))"
    cursor.execute(create_route_links)
    cursor.execute(create_trace_point)
    cursor.execute(create_file_checksum)
    cursor.execute(create_warnings)
    cursor.execute(create_warning_traffic_signs)
    total_matched_trace_length_list = []
    total_over_speed_time_list = []
    total_warnings_count_list = []
    total_warnings_text_list = []
    total_warning_sign_text_list = []
    warning_trace_points = []
    user_id = 'angela'  # YOUR USER ID (MANDATORY FOR DATABASE ENTRY)
    rootdir = './'  # ROOT PATH/FOLDER OF GPS LOGS
    for dir_path, dir_names, file_names in os.walk(rootdir):
        for file_name in file_names:
            if file_name.split('.')[-1].upper() == 'NMEA':  # APPLICABLE FORMATS: GPX/NMEA/KML/CSV
                gps_trace_file_name = os.path.join(dir_path, file_name)
                file_type = str(gps_trace_file_name).split('.')[-1].upper()
                rme_checkers(user_id, gps_trace_file_name)
    final_total_matched_trace_length = 0
    final_total_over_speed_time = 0
    final_total_warning_count = 0
    final_total_warning_sign_count = 0
    for trace_length in total_matched_trace_length_list:
        final_total_matched_trace_length += trace_length
    for over_speed_time in total_over_speed_time_list:
        final_total_over_speed_time += over_speed_time
    for warnings_count in total_warnings_count_list:
        final_total_warning_count += warnings_count
    final_over_speed_rate = round((final_total_over_speed_time / final_total_matched_trace_length) * 100, 2)
    print("final_total_matched_trace_length: {} seconds".format(final_total_matched_trace_length))
    print("final_total_over_speed_time: {} seconds".format(final_total_over_speed_time))
    print("final_over_speed_rate: {}%".format(final_over_speed_rate))
    print("final_total_warning_count: {}".format(final_total_warning_count))
    if final_total_warning_count > 0:
        print('warning:')
        for warning_text in total_warnings_text_list:
            if len(warning_text) > 0:
                print(warning_text)
    if len(total_warning_sign_text_list) > 0:
        print('passing_warning_signs: ')
        for list in total_warning_sign_text_list:
            for sublist in list:
                print('\tuser_id:{}, file_name:{}, traffic_warning_sign:{}, count:{}'.format(sublist[0], sublist[1],
                                                                                             sublist[2], sublist[3]))


filename: ./1.NMEA	|	checksum: d91b3615d7e0b33735102e107f5afb96
GPS trace is new, uploading RME...
Write result into cache database.
{"RouteLinks":[{"functionalClass":4,"confidence":0.92,"linkId":-786042585,"attributes":{"SPEED_LIMITS_FCN":[{"FROM_REF_SPEED_LIMIT":"0","TO_REF_SPEED_LIMIT":"50","SPEED_LIMIT_SOURCE":"1","SPEED_LIMIT_UNIT":"K"}]},"mSecToReachLinkFromStart":4841,"linkLength":73.5,"shape":"25.08876 121.52147 25.08874 121.52149 25.08864 121.52158 25.08854 121.52166 25.08843 121.52173 25.0883 121.52181 25.08821 121.52186"},{"functionalClass":4,"confidence":0.92,"linkId":-785969616,"attributes":{"SPEED_LIMITS_FCN":[{"FROM_REF_SPEED_LIMIT":"0","TO_REF_SPEED_LIMIT":"50","SPEED_LIMIT_SOURCE":"1","SPEED_LIMIT_UNIT":"K"}]},"mSecToReachLinkFromStart":15969,"linkLength":125.83,"shape":"25.08821 121.52186 25.0881 121.52194 25.08791 121.52206 25.08773 121.52218 25.08751 121.52231 25.08737 121.5224 25.08723 121.52249"},{"functionalClass":4,"confidence":0.94,"linkId":-1104566916,"attribu

matched trace_length: 300 seconds	overspeed_time: 27 seconds (9.0%)
0 warnings: []
-------
filename: ./2.NMEA	|	checksum: 74b3fa3d15b777d98c58e1a3036ef2b4
GPS trace is new, uploading RME...
Write result into cache database.
{"RouteLinks":[{"functionalClass":3,"confidence":0.94,"linkId":-785966157,"attributes":{"SPEED_LIMITS_FCN":[{"FROM_REF_SPEED_LIMIT":"0","TO_REF_SPEED_LIMIT":"40","SPEED_LIMIT_SOURCE":"1","SPEED_LIMIT_UNIT":"K"}]},"mSecToReachLinkFromStart":2301,"shape":"25.04468 121.53695 25.04467 121.53702 25.04466 121.53711 25.04465 121.53722 25.04464 121.53734","linkLength":39.98},{"functionalClass":3,"confidence":0.94,"linkId":-786021113,"attributes":{"SPEED_LIMITS_FCN":[{"FROM_REF_SPEED_LIMIT":"0","TO_REF_SPEED_LIMIT":"40","SPEED_LIMIT_SOURCE":"1","SPEED_LIMIT_UNIT":"K"}]},"mSecToReachLinkFromStart":4648,"shape":"25.04464 121.53734 25.04463 121.53743 25.04463 121.53751","linkLength":17.22},{"functionalClass":3,"confidence":0.9,"linkId":-1056384992,"attributes":{"SPEED_LIMITS_FC

matched trace_length: 291 seconds	overspeed_time: 0 seconds (0.0%)
1 warnings: [('angela', './2.NMEA', 15, 70, 3, 'Entering link 1063453689 at 25.04467,121.54123 into forbidden driving direction')]
-------
final_total_matched_trace_length: 591 seconds
final_total_over_speed_time: 27 seconds
final_over_speed_rate: 4.57%
final_total_warning_count: 1
[('angela', './2.NMEA', 15, 70, 3, 'Entering link 1063453689 at 25.04467,121.54123 into forbidden driving direction')]
passing_warning_signs: 
	user_id:angela, file_name:./1.NMEA, traffic_warning_sign:ACCIDENT HAZARD, count:2
	user_id:angela, file_name:./1.NMEA, traffic_warning_sign:LANE MERGING FROM THE LEFT, count:2
	user_id:angela, file_name:./1.NMEA, traffic_warning_sign:LANE MERGING FROM THE RIGHT, count:1
	user_id:angela, file_name:./2.NMEA, traffic_warning_sign:LANE MERGING FROM THE LEFT, count:2


In [2]:
import time

max_lat, min_lat, max_lon, min_lon, avg_lat, avg_lon = 0, 0, 0, 0, 0, 0

def time_to_seconds(input_time):  #'%Y-%m-%d %H:%M:%S'
    secs = int(time.mktime(time.strptime(input_time, '%Y-%m-%d %H:%M:%S'))) * 1000
    return secs

user_id = 'angela'
query_time_begins = time_to_seconds('2016-11-24 00:00:00') # Begining secs
query_time_ends = time_to_seconds('2016-11-25 00:00:00')  # Endding secs

conn = sqlite3.connect('rme_safety_checker.sqlite')
cursor = conn.cursor()

map_center_query = cursor.execute("SELECT AVG(lat_matched), AVG(lon_matched) FROM trace_points WHERE trace_points.user_id='{}' AND {} < timestamp < {}".format(user_id, query_time_begins, query_time_ends))
map_center = map_center_query.fetchall()[0]

map_bbox_query = cursor.execute("SELECT MIN(lat_matched), MIN(lon_matched), MAX(lat_matched), MAX(lon_matched) FROM trace_points WHERE trace_points.user_id='{}' AND timestamp BETWEEN {} AND {}".format(user_id, query_time_begins, query_time_ends))
map_bbox = map_center_query.fetchall()[0]

traces_query = cursor.execute("SELECT lat, lon, lat_matched, lon_matched, timestamp, heading_matched, speed_kph, from_ref_speed_limit, to_ref_speed_limit FROM trace_points INNER JOIN route_links ON trace_points.link_id_matched = route_links.link_id WHERE trace_points.user_id='{}' AND timestamp BETWEEN {} AND {}".format(user_id, query_time_begins, query_time_ends))
traces = traces_query.fetchall()

overspeed_sql = "SELECT lat, lon, lat_matched, lon_matched, timestamp, heading_matched, speed_kph, from_ref_speed_limit, to_ref_speed_limit FROM trace_points INNER JOIN route_links ON trace_points.link_id_matched = route_links.link_id WHERE (((route_direction = 'f' AND speed_kph > from_ref_speed_limit AND from_ref_speed_limit>0)) OR ((route_direction = 't' AND speed_kph > to_ref_speed_limit AND to_ref_speed_limit>0))) AND trace_points.user_id='{}' AND timestamp BETWEEN {} AND {}".format(user_id, query_time_begins, query_time_ends)
underspeed_sql = "SELECT lat, lon, lat_matched, lon_matched, timestamp, heading_matched, speed_kph, from_ref_speed_limit, to_ref_speed_limit FROM trace_points INNER JOIN route_links ON trace_points.link_id_matched = route_links.link_id WHERE (((route_direction = 'f' AND speed_kph <= from_ref_speed_limit AND from_ref_speed_limit > 0)) OR ((route_direction = 't' AND speed_kph <= to_ref_speed_limit AND to_ref_speed_limit > 0))) AND trace_points.user_id='{}' AND timestamp BETWEEN {} AND {}".format(user_id, query_time_begins, query_time_ends)

overspeed_points = cursor.execute(overspeed_sql).fetchall()
underspeed_points = cursor.execute(underspeed_sql).fetchall()

warning_sign_sql = "select distinct shape, traffic_sign_desc from warning_traffic_signs inner join route_links on warning_traffic_signs.link_ids = route_links.link_id inner join trace_points on trace_points.link_id_matched = route_links.link_id where trace_points.user_id = '{}' and timestamp BETWEEN {} AND {}".format(user_id, query_time_begins, query_time_ends)
warning_signs = cursor.execute(warning_sign_sql).fetchall()

print(len(traces), ' trace points.')

591  trace points.


In [3]:
import folium
import time

m = folium.Map(
    location = map_center,
    tiles = 'https://1.base.maps.ls.hereapi.com/maptile/2.1/maptile/newest/normal.day/{z}/{x}/{y}/256/png8?lg=cht&&apiKey=' + apikey,
    detect_retina = False,
    max_zoom = 20,
    attr =  '(c)1987-2018 HERE'
)

def draw_trace(input_trace, ori_color, matched_color):
    for point in input_trace:
        lat, lon, lat_matched, lon_matched, timestamp, heading_matched, speed_kph, from_ref_speed_limit, to_ref_speed_limit = point
        folium.PolyLine([[lat, lon], [lat_matched, lon_matched]], weight=2).add_to(m)
        folium.CircleMarker(point[0:2], radius=3, fill=True, fill_color=ori_color, fill_opacity=0.8, color=ori_color,
                                     popup=folium.Popup('timestamp: {}<br>heading: {}<br>speed_kph: {}<br>f_speed_limit: {}<br>t_speed_limit: {}'.format(time.strftime("%a, %d %b %Y %H:%M:%S", time.localtime(timestamp/1000)), heading_matched, speed_kph, from_ref_speed_limit, to_ref_speed_limit))).add_to(m)
        folium.CircleMarker(point[2:4], radius=6, fill=True, fill_color=matched_color, fill_opacity=0.8, color=matched_color,
                                     popup=folium.Popup('timestamp: {}<br>heading: {}<br>speed_kph: {}<br>f_speed_limit: {}<br>t_speed_limit: {}'.format(time.strftime("%a, %d %b %Y %H:%M:%S", time.localtime(timestamp/1000)), heading_matched, speed_kph, from_ref_speed_limit, to_ref_speed_limit))).add_to(m)
        

draw_trace(overspeed_points, '#ff99ff', '#ff1a75')
draw_trace(underspeed_points, '#66c2ff', '#4dffdb')

for warning_trace_point in warning_trace_points:
    lat = warning_trace_point['tracepoint']['latMatched']
    lon = warning_trace_point['tracepoint']['lonMatched']
    folium.map.Marker([lat, lon], popup=warning_trace_point['text'], icon=folium.Icon(color='red', icon='info-sign')).add_to(m)
    
for warning_sign in warning_signs:
    lat = float(warning_sign[0].split(' ')[0])
    lon = float(warning_sign[0].split(' ')[1])
    info = warning_sign[1]
    folium.map.Marker([lat, lon], popup=info, icon=folium.Icon(color='green', icon='warning-sign')).add_to(m)

m.fit_bounds([[map_bbox[0],map_bbox[1]],[map_bbox[2],map_bbox[3]]])

m